# Capstone Project

## Introduction/Business Problem

#####     Buenos Aires is the capital of Argentina and one of the best city to live in Latin America , it has a population of at least 2.89 million people spread in 48 neighborhoods and 13 communes, in this project we will study Buenos Aires because i've the intention of moving over the in this year.
         
#####     To move over there, first we will do an analysis  based on the cost of rent and if the area is considered safe or not, here we look for the minimum cost of rent having  the lowest crime rate possible, this is very difficult due to safe areas are expensive and their is a budget to consider.Another aspect to keep in mind is the amount of jobs sources close to the neighborhood and the ease to move around the city.

#####    A preselection will be done just based on crime rate and rent price (Cluster Analysis), then of the remaining neighborhoods we seek the ones with the most job offers and transportation methods.

## Data 

### Data Description

#### For the Map the of the neighborhoods of Buenos Aires we used the geojson data provided by the portal of the mayoralty of Buenos Aires data(https://data.buenosaires.gob.ar/dataset/barrios), the rest of the data was also extract from this portal:

#### * Rent (https://www.estadisticaciudad.gob.ar/eyc/?p=62993)

#### * Crimes (https://e-8ef01e7270.cognitiveclass.ai/download/data/delitos.csv), to handle missing data, we seek                  alternative in real state web pages. 


## Methodology

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install bokeh --yes
print("Bokeh ready!")
from bokeh.plotting import figure
from bokeh.io import output_file, show,output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool




import seaborn as sns
print('Libraries imported.')


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


^C
Bokeh ready!
Libraries imported.


## Import Datasets

In [2]:
df = pd.read_excel("/resources/data/barrios1.xlsx")# import the Neighbourhood dataset
crime = pd.read_csv("/resources/data/delitos.csv") # crime dataset



FileNotFoundError: [Errno 2] No such file or directory: '/resources/data/barrios1.xlsx'

## Exploratory Data Analysis

##### First we check the column names and the first rows of the dataset, consistency in the names of the columns is required for future merges between data frames, so if a name is not proper we change it. After calculate the amount of missing data, if is significant that column must be dropped, also other columns that wont be use in the analysis have to be drop.

In [ ]:
df.drop('Unnamed: 0',axis = 1, inplace = True) # drop unncesary columns
df.rename(columns = {"lng":"lat","lat":"lng"})
df.head()

In [ ]:
print('The dataframe has  {} neighborhoods.'.format( 
        len(df['Neighbourhood'].unique()),
        df.shape[0]))

##### Let's use the coordinates of Buenos Aires to center the map, and the coordinates of the neighborhoods to point where they are.

In [ ]:
address = 'Buenos Aires'

geolocator = Nominatim(user_agent="ba_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The  coordinates of Buenos Aires are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_ba = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neig, com  in zip(df['lat'], df['lng'],df['Neighbourhood'],df['commune']):
    label = '{}, {}'.format(neig , com)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ba)  
    
map_ba

##### Check the first rows of the crime dataset.

In [ ]:
# check the data 
crime.head()

In [ ]:
# check the NaN of these columns
print(crime[["lugar","origen_dato"]].isna().sum(),'\n', #calculate how much NaN are to drop them.
crime[["cantidad_vehiculos","cantidad_victimas"]].sum())
#Drop columns
crime.drop(["id","lugar","origen_dato","cantidad_vehiculos","cantidad_victimas","fecha"],axis = 1, inplace = True )




In [ ]:
crime.head()
crime.columns = ["commune","Neighbourhood","lat","lng","time","gun","bike","type_crime"] #rename columns

##### To know which neigbourhood is the dangerous, we group by the column "Neighbourhood" and apply the size method to count how many crimes were commited, then we rename the new column, and estimate a new column call "Pct" that is the percentage of crimes, we sort the values from most dangerus to least.

In [ ]:
# Crime per Neighbourhood
crime_Ng = pd.DataFrame(crime.groupby("Neighbourhood").size()) # count crimes in each Neighbourhood
crime_Ng.columns = ["Total"] #name column of count
crime_Ng["Pct"] = crime_Ng["Total"]/len(crime)  # pct column
crime_Ng.sort_values("Total",ascending = False,inplace = True) 
display(crime_Ng.describe(),crime_Ng.head())
crime_Ng = crime_Ng.reset_index()

In [ ]:
#Data Visuliation
fig, ax = plt.subplots(figsize=(9,5))
ax = sns.boxplot( x = "Total",data = crime_Ng)
ax.set_title("Crimes")
ax.set_xlabel("Total Crimes")
plt.show()

In [ ]:
#Data Visuliation
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(12,11))
ax =sns.barplot(y = "Neighbourhood", x = "Total",data = crime_Ng , palette = "RdBu_r")
ax.set_title("Total Crimes Per Neighbourhood")
plt.show()

##### We can see that Palermo is the most dangerous neighborhood in Buenos Aires with 27698 crimes and Puerto Madero is the least dangerous with just 338, around 90 times less than Palermo. The median, mean and standard deviation of crimes per neighborhood are 5549, 7250, and , 5905.

##### After dertiminated which are the mos dangerous and safe neighborhoods, it's important to know which type of crimes are commited the most, to do this we group by the column "type_crime" and apply the size method and sort in descending order.

In [ ]:
 #Type of Crimes
crime_ty = pd.DataFrame(crime.groupby("type_crime").size()) # count crimes in each Neighbourhood
crime_ty.columns = ["Total"] #name column of count
crime_ty["Pct"] = crime_ty["Total"]/len(crime)  # pct column
crime_ty.sort_values("Total",ascending = False,inplace = True) 
crime_ty =crime_ty.reset_index()
display(crime_ty)

In [ ]:
#Count of crimes
fig, ax = plt.subplots(figsize=(8,6))
ax = sns.barplot(y = "type_crime", x = "Total",data = crime_ty, palette = "dark")
ax.set_title("Amount of Crime per Type")
ax.set_ylabel("Type of Crime")
ax.set_xlabel("Count")
plt.show()

##### From the previous data we can see that "Theft with violence" is the most common crime and "Homicide road Safety" is the least common, we can also say that the rate of murder("Homicide" per 100000 inhabitants) in Buenos Aires is low compare to other main cities  in Latin America like Brazil, Honduras, Venezuela, Mexico, among others.

##### Finally we look at which are the most commons crimes in each neighborhood, we repeat the process of group by, in this case two columns "type_crime"and "Neighborhood" and applying the size method.

In [ ]:
#type of crime in Neighbourhood
crime_tn = pd.DataFrame(crime.groupby(["Neighbourhood","type_crime"]).size()) # count crimes in each Neighbourhood
crime_tn.columns = ["Total"] #name column of count
crime_tn.reset_index(inplace = True)
#crime_tn.head

##### To see which hour is the mos dangerous in the city, fisrt check if the column type is appropiate to work with, if isn't we convert to a date type, then extract the hour and assign it in to a list, then we create a new column call  "hour", finally we repeat the group by process and method size.

In [ ]:
type(crime.time[0]) #check the tyoe of var

In [ ]:
 time = pd.to_datetime(crime.time) #change to datetime

In [ ]:
 type(time[0]) #check change

In [ ]:
hour = []
for i in time:
    hour.append(i.hour) #extract the hour as a number for every crime

In [ ]:
crime["time"] = hour #assing the new column to the orignal DataFrame
hour = pd.DataFrame(hour) # make a Dataframe for plotting
hour.columns =["hour"] #assign name


In [ ]:
time1 = pd.DataFrame(hour.groupby("hour").size())
time1.reset_index(inplace = True)
time1.columns  = ["hour","count"]

In [ ]:
time1["pct"] = time1["count"]/len(time)
time1.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(x = "hour" ,y = "pct", ax=ax,data = time1)


ax.set_xlabel("Hour",fontsize = 14)
ax.set_title("Crimes commited per Hour",fontsize = 16)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.set_ylabel(" ",fontsize = 14)
for i in ax.patches:
    p_x = i.get_x()
    p_y = i.get_height()
    ax.annotate(xy = (p_x + 0.06, p_y + 0.001 ), 
            s = "{:.0f}%".format(round(p_y*100, 0)), 
            ha = 'left', va='center', fontsize=10)
    
    
plt.yticks([])
plt.show()
plt.show()

In [ ]:
display(time1[["hour","count"]].corr())

##### We can see that 64% of the crimes are happened in the second half of the day, and between six and 10 p.m. 32% of the crimes are committed, clearly the preferred time is by the night, when usually more people go out for distraction.

In [ ]:
#a = [1,2,3,4,5]
#for i in a:
#    fig, ax = plt.subplots(figsize=(7.5,10))
#    ax = sns.regplot(x = "hour" ,y = "count", ax=ax,data = time1,order =i)
#    ax.set_xlabel("Hour",fontsize = 14)
#    ax.set_ylabel("Count",fontsize = 16)
#    ax.set_title("Crimes per Hour, order of cruve {}".format(i),fontsize = 16)

## Import Dataset

In [ ]:
alq = pd.read_excel("/resources/data/alquileres.xlsx")

In [ ]:
alq.head()

In [ ]:
 #drop first row, and las two columns
rent = alq.iloc[2:,0:13]  #just working with data from the 2018 
rent["Barrio"] = rent["Barrio"].str.upper()
rent.describe()

In [ ]:
rent_month = pd.DataFrame(rent.mean())
rent_month["month_n"] = range(1,13) 
rent_month.columns = ["mean_rent","month_n"]
rent_month["mean_rent"] = rent_month["mean_rent"]//30 # divide to 30 to get the price per squere

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax = sns.set_style("whitegrid")
ax = sns.lineplot(x = "month_n",
                  y = "mean_rent",
                  data = rent_month,
                  color = "k")
ax = sns.scatterplot(x = "month_n",
                  y = "mean_rent",
                  data = rent_month
                    ) 
ax.set_title("Cost of the rent monthly 2018", size = 16)
ax.set_ylabel("Mean rent per month 2018", size = 14)
ax.set_xlabel("Month of the Year", size = 14)
ax.set_ylim(360,460)
ax.set_xlim(0,12)
plt.show()

In [ ]:
rent_ng = pd.DataFrame(rent.mean(axis = 1))
rent_ng["Neighbourhood"] = rent["Barrio"] 
rent_ng.columns = ["mean_rent","Neighbourhood"]
rent_ng["mean_rent"] = rent_ng["mean_rent"]//30
rent_ng = rent_ng.sort_values("mean_rent",ascending = False)
rent_ng.head()

In [ ]:
rent.rename(columns = {"Barrio":"Neighbourhood"},inplace = True)

display(rent.head(),
        rent.shape)


In [ ]:
fig, ax = plt.subplots(figsize=(15,11))
ax = sns.barplot(y = "Neighbourhood", 
            x = "mean_rent",
            data = rent_ng 
           )
ax.set_xlim(0,700)
ax.set_title("Rent Buenos Aires",size = 16)
ax.set_xlabel("Pesos per square meter",size = 14)
ax.set_ylabel("Neighbourhood",size = 14)
plt.show()

##### Here we can apreciate how all most statistical indicator have gone up from January to December, in the first chart we can see that the trend is up for all months.

##### Also we can see that Puerto Madero is by more than 200 pesos per square meter is the most exclusive neighborhood in all Buenos Aires, this area of the city is full of new and modern real state development and is one of the most popular neighborhood for turism, also happened to be one the safestest area in all the city.

##### Then we can say that the rest of the neighborhood are between 500 and 300 pesos per square meter, 22 in the range of 499 to 400 and 23 in the range of 399 to 300.

In [ ]:
crime_rent = pd.merge(crime_Ng,rent_ng, on = "Neighbourhood")
crime_rent.drop("Pct",axis=1,inplace = True) 


In [ ]:
ba_df = pd.merge(crime_rent,df ,on = "Neighbourhood")
ba_df = ba_df.rename(columns = {"Total":"Total_Crimes" ,"mean_rent":"Psm"})
display(ba_df.head())

In [ ]:
output_notebook()

In [ ]:
source = ColumnDataSource(ba_df)
p1 = figure(x_axis_label = "Price per Squre meter",y_axis_label = "Total Crimes")
p1.circle(y = 'Total_Crimes',
          x = 'Psm',
          source = source,
          size=10)
hover = HoverTool(tooltips=[
                    ('Neighbourhood','@Neighbourhood'),
                    ('Price','@Psm'),
                    ('Total Crimes','@Total_Crimes')
                    ])
p1.add_tools(hover)
output_file('cri-psm.html')
show(p1)

In [ ]:
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
drop =["commune"]#,"lat","lng"]
X = ba_df.drop(drop,axis =1).values[:,1:]

#X = np.nan_to_num(X)
cluster_dataset =Normalizer().fit_transform(X)
#cluster_dataset
ks = range(1, 9)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters = k)
    
    # Fit model to samples
    model.fit(X)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('Number of clusters, k')
plt.ylabel('Inertia')
plt.xticks(ks)
plt.show()


In [ ]:
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=20)
k_means.fit(cluster_dataset)
labels = k_means.labels_
centers = k_means.cluster_centers_
print(labels,centers)

In [ ]:
ba_df["Labels"] = labels
ba_df.head()

In [ ]:
ba_df_mean = pd.DataFrame(ba_df.groupby('Labels')[["Total_Crimes","Psm"]].mean()).reset_index()
ba_df_mean["Count"] = ba_df["Labels"].value_counts()
ba_df_mean["Labels"] = ba_df_mean["Labels"].astype('category')

display(ba_df_mean,
        ba_df_mean["Total_Crimes"].std(),
        ba_df_mean["Psm"].std())

In [ ]:
output_notebook()
source = ColumnDataSource(ba_df_mean)
 
p1 = figure(x_axis_label = "Cluster",y_axis_label = "Count")
p1.vbar(top = 'Count',
        x = 'Labels',
        source = source,
        width = 0.9)
hover = HoverTool(tooltips=[
                    ('Mean Total Crimes','@Total_Crimes'),
                    ('Mean Rent Cost','@Psm')])
p1.add_tools(hover)
output_file('mean.html')
show(p1)

## Results

##### To classify the neighborhoods, first we merge the data frames containing the crimes, rent cost and locations of each neighborhood, then we drop the column "commune", and extract the values of these columns.

##### Is necessary apply a machine learning classification method, in this case KMeans,first the "Elbow Method" was applied, and select the number where the slope start to decrease her decline the most, in this case is 5 so that will be the number of cluster to use. After that sort out the neighborhood, and attach their label to the data frame and convert to a category type.


##### To obtain the characteristic of each category, the mean method was apply:

##### Category:0 There are 15 neighborhoods, that have  a mean price per square meter of 405.67 and a mean rate crime of 6219 per year.
##### Category:1 There is 1 neighborhood, Palermo, it has  a mean price per square meter of 691 (max rent price) and a mean rate crime of 338 (min total crime) per year.
##### Category:2 There are 4 neighborhoods, that have  a mean price per square meter of 400.5 and a mean rate crime of 1526 per year.
##### Category:3 There are 9 neighborhoods, that have  a mean price per square meter of 382.22 and a mean rate crime of 3209 per year.
##### Category:4 There are 13 neighborhoods, that have  a mean price per square meter of 387.77 and a mean rate crime of 14867.15 per year.

##### The only "outlier" is Puerto Madero it has the highest rent and the lowest crime rate in all the city, then  deviation in rent price for the rest of the clusters is just 133 pesos per square meter looking to the crime stats the range is much wider than for the rent, here the deviation is 5427 crimes, so it can be said that the crime rate is the biggest difference between the clusters, and its a fundamental factor, also the rent prices  are packed in big groups, the don't differ that much from neighborhood to neighborhood.

##### Finally a representation of the category is made with an interactive bar chart, that provides the characteristics of each cluster.


In [ ]:
ba_df_2 = ba_df[ba_df.Labels == 2]
ba_df_3 = ba_df[ba_df.Labels == 3]
ba_df_23 = ba_df_2.append(ba_df_3)

In [ ]:
display(ba_df_23)

In [ ]:
dist =[]
from geopy.distance import geodesic
for lat,lng in zip(ba_df_23["lat"],ba_df_23["lng"]):
    dwtn = (-34.60389, -58.37056)
    dist.append(geodesic(dwtn,(lat,lng)).kilometers)


In [ ]:
ba_df_23["dist (km)"] = dist

In [ ]:
ba_df_23

In [ ]:
ba_df_23 = ba_df_23.sort_values("dist (km)",ascending = True).head()
ba_df_23

##### Then the cluster to be selected are the one we the lowest crime rate and price per square meter.

##### Chacarita is the clotest neighborhood to the center of the city that is ralative cheap and safe, in the top five clotest to the downtown Chacarita is the one with the most crimes and higher rent, but in this case minimize distance is the main goal so this is the neighborhood selected to move.

## Foresquare

In [ ]:
CLIENT_ID = 'YYMWR3DMQYBQSQB3WS4DJYNHBFBZRU5NYLDW3JR1MZHAURSF' # your Foursquare ID
CLIENT_SECRET = '45B3HWT33Y35RKHPXVWKDEZHRPBZCWJC1XH0BAQYFJM5DPE3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

##### Select Chacarita coordinates to extract the venues.

In [ ]:
cha_lat = ba_df_23.iloc[0,4]
cha_lng = ba_df_23.iloc[0,5]
display(cha_lat,cha_lng)

###### Look for the top 100 venues that are in Chacarita within a radius of a kilometer

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
 
radius = 1000 # define radius
 
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    cha_lat, 
    cha_lng, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
#results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
 

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

display(nearby_venues.head())

###### There are 79 venues close to Chacarita, in the next chart we can see which are the most common venues a round Chacarita.

In [ ]:
df4 = pd.DataFrame(nearby_venues.groupby("categories").size())
df4.columns = ["total"]
df4 = df4.sort_values("total",ascending = False)
df4 = df4.reset_index()

fig, ax = plt.subplots(figsize=(15,11))
ax = sns.barplot(y = "categories", 
            x = "total",
            data = df4 
           )
ax.set_xlim(0,5)
ax.set_title("Venues Categories",size = 16)
ax.set_xlabel("Total",size = 14)
ax.set_ylabel("Categories",size = 14)
plt.show()



##### The neighbourhood has all the basic store/locals that are needed pharmacy, restaurants, markers, bus stop, swimming pool among others.

In [ ]:
with open('/resources/data/barrios.geojson') as json_data:
    ba_data = json.load(json_data)
map_ba2 = folium.Map(location=[latitude, longitude], zoom_start=11)

 

map_ba2.choropleth(
    geo_data=ba_data,
    data=rent_ng,
    columns=['Neighbourhood', 'mean_rent'],
    key_on='feature.properties.BARRIO',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Rent Price per Square Meter Buenos Aires'
)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in (range(num_clusters))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ba_df['lat'], ba_df['lng'], ba_df['Neighbourhood'], ba_df['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_ba2)
       


map_ba2

In [ ]:
map_ba1 = folium.Map(location=[latitude, longitude], zoom_start=11)

with open('/resources/data/barrios.geojson') as json_data:
    ba_data = json.load(json_data)

map_ba1.choropleth(
    geo_data=ba_data,
    data=crime_Ng,
    columns=['Neighbourhood', 'Total'],
    key_on='feature.properties.BARRIO',
    fill_color='RdBu_r', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime Rate Buenos Aires'
)


# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in (range(num_clusters))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ba_df['lat'], ba_df['lng'], ba_df['Neighbourhood'], ba_df['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_ba1)
       


map_ba1

## Discussion

##### One of the aspects that stand out the most is that the average rent in Buenos Aires increased 24% the las year, one of the main reason is the financial situation in Argentina where inflation is one of the highest in the world.

##### Most of the crimes were committed between 5 to 10 pm. so the law enforcement institutions should place more platrols  at this hours in the most dangerous neighborhoods.

##### More variables can be used like public transportation, job index, or more specific matters depending on the person necessities, there are several neighborhood that have similar characteristic to Chacarita and those can be contemplated to move.

##### The rent data is a bit ambiguous because the majority of the rent required advance deposits, so the prices are a bit higher.

## Conclusion 

##### Data Analysis is a great tool in this case we could detected which are the best neighborhood to move, depending on the variables to analyze. It's important that government institutions  apply these techniques to attack problems they have in their communities like insecurity, traffic, public transportation among others.

##### Also the information should be public in this case to make decision about which neighborhoods the bes to live in or where a business can be started.